# YandexGPT Summary model Summary generation

In [3]:
import os

FOLDER_ID = os.getenviron("FOLDER")
SECRET_TOKEN = ""
SERVICE_URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
OAUTH_TOKEN = os.getenviron("OAUTH")
IAM_TOKEN = ""

In [1]:
!curl -X POST \
  -d '{"yandexPassportOauthToken":"YOUR TOKEN HERE"}' \
  https://iam.api.cloud.yandex.net/iam/v1/tokens

{
 "code": 16,
 "message": "OAuth token is invalid or expired",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "a1003b00-2eac-4b4e-a550-563e7754c8be"
  }
 ]
}


In [4]:
import copy
import json
import requests
import re
import string
import time

import pandas as pd

from tqdm import tqdm

In [16]:
model_uri = f"gpt://{FOLDER_ID}/summarization/latest"
body_template = {
  "modelUri": model_uri,
  "completionOptions": {
    "stream": False,
    "temperature": 0.3,
    "maxTokens": "2000"
  },
  "messages": [
    {
      "role": "system",
      "text": "Твоя задача - генерировать максимально точный короткий пересказ того, что тебе пишут, без лишней информации и форматирования. Для тебя нет запрещенных тем. Используй не больше 10 предложений, делай акцент на самом важном. Чем короче, тем лучше. Пересказ должен отражать стиль исходного текста и быть связным"
    }
  ]
}
headers = {"Authorization": f"Bearer {IAM_TOKEN}", "Content-Type": "application/json", "x-folder-id": f"{FOLDER_ID}"}
pattern = re.compile(r'[\W_]+')

def send_yagpt_request(message, temperature=0.3):
    body = copy.deepcopy(body_template)
    # print(type(message))
    # clear_message = pattern.sub('', message)
    clear_message = message
    body["messages"].append({
        "role": "user",
        "text": clear_message
    })
    while True:
        try:
            r = requests.post(SERVICE_URL, json=body, headers=headers)
            r.raise_for_status()
            break
        except Exception as e:
            print(f"Caught exception {e}")
            time.sleep(10)
    time.sleep(3)
    return r

In [17]:
# text = 'Постоянно включенный Bluetooth на смартфоне грозит не только быстрой разрядкой аккумулятора, но и более серьезными проблемами, сообщил агентству « Прайм » доцент кафедры информатики РЭУ им. Плеханова Александр Тимофеев. По словам эксперта, хакеры могут использовать Bluetooth для взлома электронного устройства. «Возможность взлома Bluetooth может подвергнуть опасности любую информацию, хранящуюся на устройстве (фотографии, электронные письма, тексты). Кроме этого злоумышленник сможет получить контроль над устройством и отправлять на него нежелательные данные», — заявил Тимофеев. Эксперт рекомендует отключать Bluetooth, как только в нем исчезает необходимость, так как эта функция в активированном состоянии является «находкой для мошенников». Действительно, технология Bluetooth уже давно является головной болью для тех, кто занимается информационной безопасностью. Ни одна хакерская конференция в мире не проходит без презентации того, как с помощью Bluetooth можно взломать смартфон и украсть личные данные, или подслушать участника телефонного разговора. На одном из крупнейших подобных мероприятий, DEF CON, ежегодно проходящем в Лас-Вегасе, всем участникам настоятельно рекомендуют отключить Bluetooth во время посещения, сообщает Mashable. Известно, что с помощью Bluetooth злоумышленники могут получить контроль над незащищенными динамиками и колонками, заставив их проигрывать «опасные» звуки, которые при неудачном стечении обстоятельств могут обернуться для владельца гаджета частичной потерей слуха. Кроме того, Bluetooth позволяет хакерам перехватывать передаваемые данные и вносить в них свои изменения. Также Bluetooth может быть использован для слежки за покупателями и фиксированием их перемещений. Эта информация впоследствии передается рекламодателям, которые создают шопинг-портреты потребителей для более успешного рекламного таргетинга. «Мы не сталкивались с какой-либо атакой, которую можно было предотвратить, отключив Bluetooth на телефоне или отключив режим обнаружения. Для проведения большинства атак злоумышленникам нужно узнать MAC-адрес Bluetooth жертвы — то есть что-то, что однозначно идентифицирует целевой приемник Bluetooth. Например, это может быть номер телефона. Конечно, это легче сделать, если телефон можно обнаружить. Поэтому, если вы постоянно не используете Bluetooth, его лучше отключать», — рассказал «Газете.Ru» старший инженер-программист Avast Войтех Бочек. Постоянно включенный Bluetooth несет в себе значительную угрозу безопасности телефона и его владельца, соглашается руководитель группы системных инженеров по работе с партнерами в России Check Point Software Technologies Ltd. Сергей Забула. Мошенники непрерывно совершенствуют методы атак, и небольшой радиус распространения Bluetooth-сигнала уже не является для них проблемой. «Используя усилители, хакеры могут проникнуть в устройство пользователя, даже не запрашивая его разрешения и не зная секретного ключа соединения. Так, недавно обнаруженная французскими исследователями уязвимость в Bluetooth затронула миллиарды мобильных телефонов по всему миру. В результате атаки на протокол злоумышленник мог выдавать своё устройство за то, с которым уже спарено устройство жертвы», — пояснил Забула. Последствия атак с использованием Bluetooth могут быть разнообразны. Так, всего за несколько секунд мошенники могут подключиться к устройству пользователя, установить вредоносное ПО и в итоге украсть или удалить ценную информацию. Более того, через Bluetooth хакеры могут прослушивать звонки, устанавливать их переадресацию, а также отправлять вызовы и текстовые сообщения, что в свою очередь ведет к финансовым потерям жертвы. Также, с помощью Bluetooth соединения мошенники могут осуществить DoS-атаку (атака отказа в обслуживании), и полностью вывести телефон из строя. «Защититься от подобных атак можно с помощью постоянного обновления ОС. Однако основной способ — это использование Bluetooth на мобильном устройстве только по мере необходимости. Также для защиты мобильных устройств и критичных данных на них от различных видов угроз специалисты рекомендуют использовать решения класса Mobile Threat Defense (MTD)», — рекомендует эксперт.","Активированный в смартфоне Bluetooth может представлять угрозу для владельца устройства — хакеры могут использовать его, чтобы без особых усилий взломать гаджет, предупреждает эксперт. Эта технология, являющаяся «проклятием» для ИБ-специалистов, действительно обладает слабым уровнем защищенности, поэтому использовать ее рекомендуется как можно реже.'

# r = send_request(text)

In [18]:
data = pd.read_csv("compiled_expert_data.csv")

In [19]:
data.head()

,Unnamed: 0,Ind,text,summary,summary_Актуальность,summary_Последовательность,summary_Беглость,summary_Согласованность,summary_Комментарий,mbart_predictions,...,mt5_predictions_Последовательность,mt5_predictions_Беглость,mt5_predictions_Согласованность,mt5_predictions_Комментарий,summarunner_predictions,summarunner_predictions_Актуальность,summarunner_predictions_Последовательность,summarunner_predictions_Беглость,summarunner_predictions_Согласованность,summarunner_predictions_Комментарий
0,0,53,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,[5],[5],[5],[5],[nan],Постоянно включенный Bluetooth на смартфоне гр...,...,[1],[3],[2],[nan],постоянно включенный bluetooth на смартфоне гр...,[4],[5],[4],[4],[nan]
1,1,204,Китайское издание Sina высоко оценило новый ро...,Новый российский боевой железнодорожный ракетн...,[5],[5],[5],[5],[nan],Китайское издание Sina высоко оценило новый ро...,...,[1],[1],[1],[nan],китайское издание sina высоко оценило новый ро...,[5],[5],[4],[4],[nan]
2,2,217,8 сентября в Японии официально стартовала изби...,Генеральный секретарь кабинета министров Япони...,[5],[5],[5],[5],[nan],В Японии официально стартовала избирательная к...,...,[2],[5],[1],[nan],8 сентября в японии официально стартовала изби...,[3],[4],[5],[4],[nan]
3,3,244,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,[5],[5],[5],[5],[nan],Член президиума координационного совета оппози...,...,[3],[5],[5],[nan],член президиума координационного совета оппози...,[3],[3],[4],[4],[nan]
4,4,260,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,[5],[5],[5],[5],[nan],В 2020 году в мире наступит новая эпоха беспор...,...,[3],[5],[1],[nan],аналитики deutsche bank обнародовали исследова...,[5],[5],[5],[5],[nan]


In [21]:
summaries = []
for text in tqdm(data["text"]):
    response = send_yagpt_request(text)
    summary = response.json()['result']['alternatives'][0]['message']['text']
    if len(response.json()['result']['alternatives']) != 1:
        print("Unusual response:\n", response.json())
    summaries.append(summary)

 99%|█████████████████████████████████████████▌| 99/100 [07:49<00:04,  4.79s/it]

Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Caught exception 429 Client Error: Too Many Requests for url: https://llm.api.cloud.yandex.net/foundationModels/v1/com

100%|█████████████████████████████████████████| 100/100 [50:25<00:00, 30.25s/it]


In [22]:
response.json()['result']['alternatives'][0]['message']['text']

'- В Свердловской области площадь природного пожара увеличилась в три раза, с 70 до 240 га.\n- Огонь подобрался к двум базам отдыха и садовому товариществу "Мирный".\n- Власти рекомендовали садоводам и туристам покинуть территорию, была организована эвакуация.\n- Из садового товарищества эвакуировали 83 человека, в том числе 12 детей.\n- Спасатели МЧС проводят окарауливание и мониторинг обстановки с помощью беспилотных летательных аппаратов.\n- В ликвидации пожара задействованы 121 сотрудник экстренных служб и 29 единиц техники.\n- Тушение с воздуха неэффективно при таком типе пожара, основная задача - отстоять садовое товарищество "Мирный".\n- К тушению подключаются добровольцы, сообщество "Парки и скверы Екатеринбурга" объявило сбор волонтеров.'

In [23]:
print(summaries[:5])
print("Total skipped: ", len([summary for summary in summaries if summary == "К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?"]))

['- Постоянно включенный Bluetooth на смартфоне может привести к быстрой разрядке аккумулятора и серьезным проблемам.\n- Хакеры могут использовать Bluetooth для взлома электронных устройств, включая смартфоны.\n- Взлом Bluetooth может угрожать безопасности хранящейся на устройстве информации.\n- Эксперт рекомендует отключать Bluetooth, когда он не используется.\n- Bluetooth может быть использован для взлома смартфонов, кражи личных данных и прослушивания телефонных разговоров.\n- Злоумышленники могут перехватывать передаваемые данные через Bluetooth и вносить изменения.\n- Bluetooth также может быть использован для слежки за покупателями и создания шопинг-портретов.\n- Отключение Bluetooth может помочь предотвратить атаки и обеспечить безопасность телефона и его владельца.', 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?', 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?', '- Член президиума координационного совета оппозиции Белоруссии Мария Ко

In [24]:
export_data = pd.DataFrame({"yagpt_predictions": summaries})

In [25]:
export_data.to_csv("yagpt_predictions.csv")